In [ ]:
import sys
!{sys.executable} -m pip install torch diffusers accelerate scipy ftfy safetensors transformers numpy

In [ ]:
import torch
from diffusers import AnimateDiffPipeline, DDIMScheduler, MotionAdapter
from diffusers.utils import export_to_gif

MOTION_ADAPTOR = "guoyww/animatediff-motion-adapter-v1-5-2"
#MODEL_ID = "SG161222/Realistic_Vision_V5.1_noVAE"
MODEL_ID = 'frankjoshua/toonyou_beta6'
#MODEL_ID = 'Yntec/ResidentCNZCartoon3D'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch_dtype = torch.float16 if device.type == "cuda" else torch.float32

# Load the motion adapter
adapter = MotionAdapter.from_pretrained(MOTION_ADAPTOR, torch_dtype=torch_dtype).to(device)

# load SD 1.5 based finetuned model

pipe = AnimateDiffPipeline.from_pretrained(MODEL_ID, motion_adapter=adapter, torch_dtype=torch_dtype).to(device)

scheduler = DDIMScheduler.from_pretrained(
    MODEL_ID,
    subfolder="scheduler",
    clip_sample=False,
    timestep_spacing="linspace",
    beta_schedule="linear",
    steps_offset=1,
)
pipe.scheduler = scheduler

# enable memory savings
pipe.enable_vae_slicing()

if device.type == "cuda":
    pipe.enable_model_cpu_offload()



In [ ]:

def make_gif(prompt, path, seconds=2, batch_count=4, seed=42):
    output = pipe(
        prompt=prompt,
        negative_prompt="bad quality, worse quality",
        num_frames=8 * seconds,
        guidance_scale=7.5,
        num_inference_steps=25,
        generator=torch.Generator("cpu").manual_seed(seed),
        width=480,
        height=272,
        num_images_per_prompt=batch_count,
    )
    frames = output.frames[0]
    export_to_gif(frames, path)
        
    
prompts = [
  "(masterpiece, best quality), 1girl, wearing a banana costume, playful pose, bright yellow outfit with banana peel details, short wavy hair, looking at viewer, cheerful expression, tropical background, sunlight filtering through leaves, detailed costume texture, vibrant colors, smile, happy",
  
  "(masterpiece, best quality), 1girl, wearing a papaya-themed dress, orange and green outfit with papaya seed pattern, elegant flowing fabric, long curly hair, standing in a tropical garden, soft glowing sunlight, warm color palette, looking at viewer, smile, happy, dynamic pose",
  
  "(masterpiece, best quality), 1girl, dressed in a watermelon-themed outfit, red dress with black seed patterns, green leafy hat, short bob hair, sitting on a picnic blanket, summer vibes, bright and cheerful, looking at viewer, smile, happy, soft breeze, detailed shading",
  
  "(masterpiece, best quality), 1girl, pineapple costume, cute ruffled dress with pineapple texture, leafy crown headpiece, curly blonde hair, standing in a tropical beach setting, ocean waves in the background, warm golden sunlight, looking at viewer, smile, happy, vibrant and colorful",
  
  "(masterpiece, best quality), 1girl, apple-inspired dress, deep red outfit with green leaf accents, long straight hair, soft freckles, autumn forest background, warm golden hues, slight breeze moving hair, looking at viewer, smile, happy, intricate details, cozy atmosphere",
  
  "(masterpiece, best quality), 1girl, strawberry-themed costume, red dress with small white polka dots, green leafy collar, short twin tails, bright and cheerful, standing in a field of strawberries, warm sunlight, dynamic pose, looking at viewer, smile, happy, soft pink hues",
  
  "(masterpiece, best quality), 1girl, kiwi-inspired outfit, green and brown dress with fuzzy texture, detailed kiwi slice patterns, short wavy hair, standing in a fantasy orchard, glowing green ambiance, looking at viewer, smile, happy, whimsical and playful, vibrant lighting",
  
  "(masterpiece, best quality), 1girl, cherry-themed outfit, red dress with twin cherries hanging from the belt, long straight dark hair, standing near a cherry blossom tree, petals falling gently, romantic atmosphere, detailed costume, looking at viewer, smile, happy, soft pink hues"
]

for idx, prompt in enumerate(prompts):
    print(idx)
    make_gif(prompt, '/workspace/v4_%02d_batch1.gif' % (idx+1), seed=42)
    make_gif(prompt, '/workspace/v4_%02d_batch2.gif' % (idx+1), seed=41)
    make_gif(prompt, '/workspace/v4_%02d_batch3.gif' % (idx+1), seed=40)
    make_gif(prompt, '/workspace/v4_%02d_batch4.gif' % (idx+1), seed=39)

In [ ]:
!tar -czvf /workspace/gifs.tar.gz /workspace/*.gif